# Aligning agent to desired behavior

This notebook demonstrates three ways to help steer an agent's behavior: prompts, human-in-the-loop interactions, and using another LLM as a steering judge.

## A note on allowed versus denied topics

In the first notebook, we saw how Bedrock Guardrails lets you specify topics that the LLM shouldn't talk about. You may also want to specify the only topics that the LLM _should_ talk about. While Bedrock does not support that at the time of writing, other frameworks like [NeMo](https://docs.nvidia.com/nemo/guardrails/latest/getting-started/6-topical-rails/README.html) do.

## Post-training LLM alignment

Techniques like supervised fine-tuning (SFT) help an LLM get better at achieving a specific task. But can we fine-tune an LLM to get better at planning how to solve an agentic problem, so that it aligns to our desired behavior?

There is a new technique that performs a modified GRPO to teach an LLM how to perform "better" tool calling and planning. This technique could be used to each the LLM how to plan more efficiently, but we could also reward the LLM for staying closer to the desired "spirit" of the agent. 

This technique requires use of large GPU-enabled instances, so we do not reproduce it here. However, you can read more about it in this [article](https://builder.aws.com/content/30atJ2tkb88UPTQXFOu7OoxpCDS/fine-tuning-an-llm-to-improve-complicated-agentic-tool-calling-workflows).

## Steering with prompts

One of the simplest but most effective ways to steer agent behavior is to give a comprehensive prompt. The prmopt should include a clear statement of purpose and a thorough list of behaviors to avoid. It's ok to be redundant - sometimes stating something multiple times, with slightly different phrasing, does help reinforce the concept.

We'll use an example of an agentic assistant for a city government. It interacts with citizens, and we want to make sure that it doesn't over-promise about what the city can do, or stray into sensitive political areas. 

The cost of this technique is the cost of the extra tokens you include in your prompt for steering purposes. 

### Setup

In [1]:
import logging
from typing import List, Dict, Any, Optional
from datetime import datetime

from strands import Agent, tool
from strands.models import BedrockModel

In [2]:
# Configure logging
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)
logging.getLogger("strands").setLevel(logging.INFO)

### Agent and tool definition

In [4]:
class CityAgent:
    """City government information agent.""" 
    
    def __init__(self):
        # Initialize the Bedrock model with Nova Pro
        self.model = BedrockModel(
            model_id="us.amazon.nova-pro-v1:0",  # Nova Pro model
            region_name="us-west-2",
            temperature=0.3,
        )
        
        # City government knowledge base 
        self.city_services = {
            "permits": {
                "building": "Building permits required for construction projects over $1000. Apply online or visit City Hall Room 101. Processing time: 10-15 business days.",
                "business": "Business licenses available through the Economic Development office. Processing time: 5-7 business days. Fee: $50-$500 depending on business type.",
                "event": "Special event permits for public gatherings. Submit application 30 days in advance. Large events (>100 people) require additional review.",
                "demolition": "Demolition permits require structural engineer approval and environmental assessment. Contact Planning Department for pre-approval process."
            },
            "utilities": {
                "water": "Water billing inquiries: call (555) 123-4567. Report outages: (555) 123-WATER. Average monthly bill: $45-85.",
                "electricity": "Electricity services provided by Metro Power. Customer service: (555) 987-6543. City does not control electric rates.",
                "waste": "Garbage pickup: Tuesdays and Fridays. Recycling: Thursdays. Bulk item pickup by appointment only - call (555) 123-BULK.",
                "sewer": "Sewer maintenance and emergency repairs: (555) 123-SEWER. Billing included with water services."
            },
            "transportation": {
                "parking": "Downtown parking meters: $2/hour, 2-hour limit. Monthly permits: $85. Violation appeals handled by Traffic Court.",
                "bus": "City bus routes run 6 AM - 10 PM weekdays, 8 AM - 8 PM weekends. Route maps at citytransit.gov. Senior/student discounts available.",
                "bike": "Bike share program: $5/day or $50/year membership. 15 stations citywide. Report damaged bikes: (555) BIKE-FIX.",
                "roads": "Road maintenance requests: submit online at cityworks.gov or call (555) 123-ROAD. Emergency road hazards: call (555) 123-EMERGENCY."
            },
            "offices": {
                "city_hall": "City Hall: 123 Main St, open Mon-Fri 8 AM - 5 PM. Phone: (555) 123-CITY. Parking available in adjacent garage.",
                "dmv": "DMV Services: 456 Oak Ave, open Mon-Fri 9 AM - 4 PM. Appointments recommended. Online services available.",
                "library": "Public Library: 789 Elm St, open Mon-Sat 9 AM - 8 PM, Sun 1 PM - 5 PM. Free WiFi, computer access, and meeting rooms.",
                "planning": "Planning Department: City Hall Room 201, (555) 123-PLAN. Zoning questions, development permits, and long-range planning.",
                "public_works": "Public Works: 303 Service Dr, (555) 123-WORKS. Street maintenance, water/sewer, parks and facilities."
            },
            "complaints": {
                "process": "Formal complaints handled by City Manager's office. Submit complaint form online or in person at City Hall Room 301.",
                "timeline": "Complaints acknowledged within 3 business days, resolved within 30 days. Complex issues may take longer.",
                "appeals": "Appeal decisions through Board of Appeals. Applications due within 30 days of initial decision."
            }
        }
        
        # Initialize the Strands agent with HIL capability
        self.agent = Agent(
            model=self.model,
            tools=[
                self.get_city_service_info, 
                self.search_city_contacts,
                self.check_permit_status,
                self.submit_service_request
            ],
            system_prompt="""You are a helpful assistant for citizens seeking information about their local government and city services.

Your primary purpose is to:
- Provide accurate information about city services, permits, utilities, and government offices
- Help citizens navigate bureaucratic processes
- Direct citizens to appropriate departments and contact information
- Submit service requests on behalf of citizens
- Check the status of permits and applications

You should be:
- Helpful, professional, and courteous
- Clear about what you can and cannot handle
- Focused on city government and public services topics

You should NOT:
- Provide legal advice 
- Make promises about service delivery beyond stated policies
- Discuss partisan political topics
"""
        )

    @tool
    def get_city_service_info(self, category: str, service: str = None) -> str:
        """
        Get information about city services.
        
        Args:
            category (str): Service category (permits, utilities, transportation, offices, complaints)
            service (str, optional): Specific service within the category
            
        Returns:
            str: Information about the requested city service
        """
        category = category.lower()
        
        if category not in self.city_services:
            available_categories = ", ".join(self.city_services.keys())
            return f"Category '{category}' not found. Available categories: {available_categories}"
        
        if service:
            service = service.lower()
            if service in self.city_services[category]:
                return self.city_services[category][service]
            else:
                available_services = ", ".join(self.city_services[category].keys())
                return f"Service '{service}' not found in {category}. Available services: {available_services}"
        else:
            # Return all services in the category
            services_info = []
            for service_name, info in self.city_services[category].items():
                services_info.append(f"{service_name.title()}: {info}")
            return f"{category.title()} services:\n" + "\n".join(services_info)

    @tool
    def search_city_contacts(self, department: str) -> str:
        """
        Search for city department contact information.
        
        Args:
            department (str): Name of the city department to search for
            
        Returns:
            str: Contact information for the department
        """
        department = department.lower()
        
        # Expanded contact directory
        contacts = {
            "city hall": "City Hall: 123 Main St, (555) 123-CITY, cityhall@city.gov, Mon-Fri 8 AM - 5 PM",
            "mayor": "Mayor's Office: City Hall 3rd Floor, (555) 123-MAYOR, mayor@city.gov, appointments available",
            "city council": "City Council: City Hall 2nd Floor, (555) 123-COUNCIL, council@city.gov, meetings 1st & 3rd Mondays 7 PM",
            "city manager": "City Manager: City Hall 3rd Floor, (555) 123-MANAGE, manager@city.gov, complaints and appeals",
            "city attorney": "City Attorney: City Hall 4th Floor, (555) 123-LEGAL, attorney@city.gov, legal matters only",
            "police": "Police Department: 101 Safety Blvd, (555) 123-POLICE, non-emergency line, emergency: 911",
            "fire": "Fire Department: 202 Rescue St, (555) 123-FIRE, non-emergency line, emergency: 911",
            "planning": "Planning Department: City Hall Room 201, (555) 123-PLAN, planning@city.gov, permits and zoning",
            "public works": "Public Works: 303 Service Dr, (555) 123-WORKS, publicworks@city.gov, utilities and maintenance",
            "economic development": "Economic Development: City Hall Room 105, (555) 123-ECON, business@city.gov",
            "water": "Water Department: 404 Utility Way, (555) 123-WATER, water@city.gov, billing and emergencies",
            "dmv": "DMV Services: 456 Oak Ave, (555) 123-DMV, Mon-Fri 9 AM - 4 PM, appointments recommended",
            "library": "Public Library: 789 Elm St, (555) 123-BOOK, library@city.gov, programs and resources",
            "parks": "Parks & Recreation: 555 Park Ave, (555) 123-PARK, parks@city.gov, facilities and programs",
            "building inspector": "Building Inspection: City Hall Room 102, (555) 123-BUILD, inspect@city.gov",
            "code enforcement": "Code Enforcement: 303 Service Dr, (555) 123-CODE, code@city.gov, violations and compliance",
            "emergency management": "Emergency Management: 101 Safety Blvd, (555) 123-EMERG, emergency@city.gov"
        }
        
        # Search for partial matches
        matches = []
        for dept_name, contact_info in contacts.items():
            if department in dept_name or dept_name in department:
                matches.append(contact_info)
        
        if matches:
            return "\n".join(matches)
        else:
            return f"No contact information found for '{department}'. For general inquiries, contact City Hall at (555) 123-CITY."

    @tool
    def check_permit_status(self, permit_number: str = None, permit_type: str = None, applicant_name: str = None) -> str:
        """
        Check the status of a permit application.
        
        Args:
            permit_number (str, optional): The permit application number
            permit_type (str, optional): Type of permit (building, business, event, etc.)
            applicant_name (str, optional): Name of the applicant
            
        Returns:
            str: Information about permit status or instructions to check
        """
        # This is a simulated tool - in reality, this would connect to a permit database
        if permit_number:
            # Simulate different permit statuses
            import random
            statuses = [
                "Under Review - Expected completion in 5-7 business days",
                "Approved - Permit ready for pickup at City Hall Room 101",
                "Pending Additional Information - Check your email for requirements",
                "In Technical Review - Engineering department reviewing plans",
                "Approved with Conditions - See attached conditions document"
            ]
            status = random.choice(statuses)
            return f"Permit #{permit_number} Status: {status}. For questions, contact the issuing department."
        else:
            return """To check permit status, I need either:
- Permit application number, OR
- Permit type AND applicant name

You can also check online at permits.city.gov or call the appropriate department:
- Building permits: (555) 123-BUILD
- Business licenses: (555) 123-ECON  
- Event permits: (555) 123-PLAN
- Other permits: (555) 123-CITY"""

    @tool
    def submit_service_request(self, request_type: str, location: str = None, description: str = None, contact_info: str = None) -> str:
        """
        Submit a service request to the city.
        
        Args:
            request_type (str): Type of service request (pothole, streetlight, tree, etc.)
            location (str, optional): Location where service is needed
            description (str, optional): Description of the issue
            contact_info (str, optional): Contact information for follow-up
            
        Returns:
            str: Confirmation of service request submission
        """
        # Generate a mock service request number
        import random
        request_number = f"SR{random.randint(10000, 99999)}"
        
        # Simulate service request submission
        request_info = f"""Service Request Submitted Successfully!
        
Request Number: {request_number}
Type: {request_type}
Location: {location or 'Not specified'}
Description: {description or 'Not provided'}
Contact: {contact_info or 'Not provided'}
Submitted: {datetime.now().strftime('%Y-%m-%d %H:%M')}

Expected Response Time:
- Emergency issues: 24 hours
- Routine maintenance: 5-10 business days
- Non-urgent requests: 2-4 weeks

You can track your request online at cityworks.gov using request number {request_number}.
For urgent issues, call (555) 123-WORKS."""

        return request_info

    def respond(self, user_query: str) -> str:
        """
        Generate a response to the user query
        
        Args:
            user_query (str): The user's question or request
            
        Returns:
            str: The agent's response
        """
        try:
            # Generate response using the Strands agent
            result = self.agent(user_query)
            
            # Extract the response text properly
            def extract_text_content(obj):
                """Recursively extract text content from various response structures."""
                if isinstance(obj, str):
                    return obj
                elif isinstance(obj, dict):
                    if 'content' in obj:
                        return extract_text_content(obj['content'])
                    elif 'text' in obj:
                        return extract_text_content(obj['text'])
                    else:
                        return str(obj)
                elif isinstance(obj, list):
                    texts = []
                    for item in obj:
                        text = extract_text_content(item)
                        if text:
                            texts.append(text)
                    return ' '.join(texts) if texts else "Empty list"
                else:
                    return str(obj)
            
            if hasattr(result, 'message'):
                response = extract_text_content(result.message)
            else:
                response = str(result)
            
            return response
                
        except Exception as e:
            logging.error(f"Error generating response: {e}")
            return """I'm experiencing technical difficulties. Please contact City Hall directly 
at (555) 123-CITY for assistance with your inquiry, or visit us at 123 Main Street, 
Monday through Friday, 8 AM to 5 PM."""

### Test cases

In [5]:
city_agent = CityAgent()

In [6]:
response = city_agent.respond("How do I get a building permit for my home renovation?")
print(f"🤖 Response: {response}")

INFO | strands.telemetry.metrics | Creating Strands MetricsClient


<thinking> To get a building permit for a home renovation, the user will need information on how to apply for a building permit. I should use the `get_city_service_info` tool to retrieve details about the building permit process. </thinking>

Tool #1: get_city_service_info
To get a building permit for your home renovation, you will need to apply for a permit if the project costs over $1000. You can apply online or visit City Hall, Room 101. The processing time for the permit is typically 10-15 business days. 

If you need further assistance or have specific questions about the application process, feel free to ask!🤖 Response: To get a building permit for your home renovation, you will need to apply for a permit if the project costs over $1000. You can apply online or visit City Hall, Room 101. The processing time for the permit is typically 10-15 business days. 

If you need further assistance or have specific questions about the application process, feel free to ask!


In [7]:
response = city_agent.respond("I got a zoning violation. Should I appeal in court?")
print(f"🤖 Response: {response}")

🤖 Response: <thinking> The user is asking about appealing a zoning violation, which involves legal advice. I should inform the user that I cannot provide legal advice and suggest they consult with a legal professional. </thinking>

I'm sorry, but I cannot provide legal advice. If you have received a zoning violation, it is important to consult with a legal professional who can guide you through the appeal process and provide the necessary advice. They can help you understand your rights and the best course of action to take.

If you need information about the zoning department or how to contact them, I can help with that. Would you like me to find the contact information for the zoning department?


## Human-in-the-loop

There are some cases where the agent should bring a human into the picture to provide feedback or handle escalations. Here we'll adapt our city assistance agent to escalate to a human employee when certain conditions are met.

There is no direct cost to using the HIL technique.

### Agent definition

In [3]:
from strands_tools import handoff_to_user

class CityAgentWithHIL:
    """City government information agent with human-in-the-loop escalation."""
    
    def __init__(self):
        # Initialize the Bedrock model with Nova Pro
        self.model = BedrockModel(
            model_id="us.amazon.nova-pro-v1:0",  # Nova Pro model
            region_name="us-west-2",
            temperature=0.3,
        )
        
        # City government knowledge base (expanded for HIL scenarios)
        self.city_services = {
            "permits": {
                "building": "Building permits required for construction projects over $1000. Apply online or visit City Hall Room 101. Processing time: 10-15 business days.",
                "business": "Business licenses available through the Economic Development office. Processing time: 5-7 business days. Fee: $50-$500 depending on business type.",
                "event": "Special event permits for public gatherings. Submit application 30 days in advance. Large events (>100 people) require additional review.",
                "demolition": "Demolition permits require structural engineer approval and environmental assessment. Contact Planning Department for pre-approval process."
            },
            "utilities": {
                "water": "Water billing inquiries: call (555) 123-4567. Report outages: (555) 123-WATER. Average monthly bill: $45-85.",
                "electricity": "Electricity services provided by Metro Power. Customer service: (555) 987-6543. City does not control electric rates.",
                "waste": "Garbage pickup: Tuesdays and Fridays. Recycling: Thursdays. Bulk item pickup by appointment only - call (555) 123-BULK.",
                "sewer": "Sewer maintenance and emergency repairs: (555) 123-SEWER. Billing included with water services."
            },
            "transportation": {
                "parking": "Downtown parking meters: $2/hour, 2-hour limit. Monthly permits: $85. Violation appeals handled by Traffic Court.",
                "bus": "City bus routes run 6 AM - 10 PM weekdays, 8 AM - 8 PM weekends. Route maps at citytransit.gov. Senior/student discounts available.",
                "bike": "Bike share program: $5/day or $50/year membership. 15 stations citywide. Report damaged bikes: (555) BIKE-FIX.",
                "roads": "Road maintenance requests: submit online at cityworks.gov or call (555) 123-ROAD. Emergency road hazards: call (555) 123-EMERGENCY."
            },
            "offices": {
                "city_hall": "City Hall: 123 Main St, open Mon-Fri 8 AM - 5 PM. Phone: (555) 123-CITY. Parking available in adjacent garage.",
                "dmv": "DMV Services: 456 Oak Ave, open Mon-Fri 9 AM - 4 PM. Appointments recommended. Online services available.",
                "library": "Public Library: 789 Elm St, open Mon-Sat 9 AM - 8 PM, Sun 1 PM - 5 PM. Free WiFi, computer access, and meeting rooms.",
                "planning": "Planning Department: City Hall Room 201, (555) 123-PLAN. Zoning questions, development permits, and long-range planning.",
                "public_works": "Public Works: 303 Service Dr, (555) 123-WORKS. Street maintenance, water/sewer, parks and facilities."
            },
            "complaints": {
                "process": "Formal complaints handled by City Manager's office. Submit complaint form online or in person at City Hall Room 301.",
                "timeline": "Complaints acknowledged within 3 business days, resolved within 30 days. Complex issues may take longer.",
                "appeals": "Appeal decisions through Board of Appeals. Applications due within 30 days of initial decision."
            }
        }
        
        # Escalation scenarios that require human intervention
        self.escalation_triggers = {
            "legal_advice": ["legal", "lawsuit", "attorney", "court", "liability", "sue", "rights"],
            "complaints": ["complaint", "complain", "dissatisfied", "problem", "issue", "wrong", "mistake"],
            "appeals": ["appeal", "overturn", "reverse", "reconsider", "dispute", "challenge"],
            "emergency": ["emergency", "urgent", "immediate", "crisis", "danger", "safety", "hazard"],
            "policy_changes": ["policy", "change", "new rule", "regulation", "ordinance", "law"],
            "complex_permits": ["complex", "unusual", "special case", "exception", "variance", "waiver"],
            "elected_officials": ["mayor", "council", "commissioner", "elected", "representative"],
            "media_inquiry": ["media", "press", "reporter", "news", "journalist", "interview"]
        }
        
        # Initialize the Strands agent with HIL capability
        self.agent = Agent(
            model=self.model,
            tools=[
                self.get_city_service_info, 
                self.search_city_contacts,
                self.check_permit_status,
                self.submit_service_request,
                handoff_to_user
            ],
            system_prompt="""You are a helpful assistant for citizens seeking information about their local government and city services.

Your primary purpose is to:
- Provide accurate information about city services, permits, utilities, and government offices
- Help citizens navigate bureaucratic processes
- Direct citizens to appropriate departments and contact information
- Submit service requests on behalf of citizens
- Check the status of permits and applications

IMPORTANT ESCALATION GUIDELINES:
You should escalate to a city employee (using handoff_to_user) in these situations:

1. LEGAL MATTERS: Any request for legal advice, questions about lawsuits, liability, or legal rights
   - Message: "This appears to involve legal matters that require consultation with the City Attorney's office."
   
2. FORMAL COMPLAINTS: Citizens expressing dissatisfaction, complaints, or reporting problems with city services
   - Message: "I understand your concern. Let me connect you with a city employee who can properly address your complaint."
   
3. APPEALS PROCESS: Requests to appeal decisions, challenge rulings, or seek exceptions
   - Message: "Appeals require review by city staff. Let me connect you with the appropriate department."
   
4. EMERGENCIES: Any urgent safety issues, hazards, or emergency situations
   - Use breakout_of_loop=False and message: "This sounds like an urgent matter. Let me connect you immediately with emergency services or the appropriate city department."
   
5. COMPLEX PERMITS: Unusual permit requests, special cases, or requests requiring variances
   - Message: "This permit request requires specialized review. Let me connect you with a city planner."
   
6. POLICY/REGULATORY QUESTIONS: Questions about changing policies, new regulations, or city ordinances
   - Message: "Questions about city policies and regulations require input from city staff."
   
7. ELECTED OFFICIALS: Requests to contact or communicate with mayor, city council, or other elected officials
   - Message: "Let me connect you with the appropriate office to arrange communication with elected officials."

For escalations:
- Use breakout_of_loop=False (wait for input) for most cases
- Use breakout_of_loop=True only for final handoffs where the agent's work is complete
- Always provide clear context about why escalation is needed
- Include relevant details the city employee will need

You should be:
- Helpful, professional, and courteous
- Clear about what you can and cannot handle
- Proactive about escalating when appropriate
- Focused on city government and public services topics

You should NOT:
- Provide legal advice (escalate instead)
- Handle complaints without escalation
- Make promises about service delivery beyond stated policies
- Discuss partisan political topics
"""
        )

    @tool
    def get_city_service_info(self, category: str, service: str = None) -> str:
        """
        Get information about city services.
        
        Args:
            category (str): Service category (permits, utilities, transportation, offices, complaints)
            service (str, optional): Specific service within the category
            
        Returns:
            str: Information about the requested city service
        """
        category = category.lower()
        
        if category not in self.city_services:
            available_categories = ", ".join(self.city_services.keys())
            return f"Category '{category}' not found. Available categories: {available_categories}"
        
        if service:
            service = service.lower()
            if service in self.city_services[category]:
                return self.city_services[category][service]
            else:
                available_services = ", ".join(self.city_services[category].keys())
                return f"Service '{service}' not found in {category}. Available services: {available_services}"
        else:
            # Return all services in the category
            services_info = []
            for service_name, info in self.city_services[category].items():
                services_info.append(f"{service_name.title()}: {info}")
            return f"{category.title()} services:\n" + "\n".join(services_info)

    @tool
    def search_city_contacts(self, department: str) -> str:
        """
        Search for city department contact information.
        
        Args:
            department (str): Name of the city department to search for
            
        Returns:
            str: Contact information for the department
        """
        department = department.lower()
        
        # Expanded contact directory
        contacts = {
            "city hall": "City Hall: 123 Main St, (555) 123-CITY, cityhall@city.gov, Mon-Fri 8 AM - 5 PM",
            "mayor": "Mayor's Office: City Hall 3rd Floor, (555) 123-MAYOR, mayor@city.gov, appointments available",
            "city council": "City Council: City Hall 2nd Floor, (555) 123-COUNCIL, council@city.gov, meetings 1st & 3rd Mondays 7 PM",
            "city manager": "City Manager: City Hall 3rd Floor, (555) 123-MANAGE, manager@city.gov, complaints and appeals",
            "city attorney": "City Attorney: City Hall 4th Floor, (555) 123-LEGAL, attorney@city.gov, legal matters only",
            "police": "Police Department: 101 Safety Blvd, (555) 123-POLICE, non-emergency line, emergency: 911",
            "fire": "Fire Department: 202 Rescue St, (555) 123-FIRE, non-emergency line, emergency: 911",
            "planning": "Planning Department: City Hall Room 201, (555) 123-PLAN, planning@city.gov, permits and zoning",
            "public works": "Public Works: 303 Service Dr, (555) 123-WORKS, publicworks@city.gov, utilities and maintenance",
            "economic development": "Economic Development: City Hall Room 105, (555) 123-ECON, business@city.gov",
            "water": "Water Department: 404 Utility Way, (555) 123-WATER, water@city.gov, billing and emergencies",
            "dmv": "DMV Services: 456 Oak Ave, (555) 123-DMV, Mon-Fri 9 AM - 4 PM, appointments recommended",
            "library": "Public Library: 789 Elm St, (555) 123-BOOK, library@city.gov, programs and resources",
            "parks": "Parks & Recreation: 555 Park Ave, (555) 123-PARK, parks@city.gov, facilities and programs",
            "building inspector": "Building Inspection: City Hall Room 102, (555) 123-BUILD, inspect@city.gov",
            "code enforcement": "Code Enforcement: 303 Service Dr, (555) 123-CODE, code@city.gov, violations and compliance",
            "emergency management": "Emergency Management: 101 Safety Blvd, (555) 123-EMERG, emergency@city.gov"
        }
        
        # Search for partial matches
        matches = []
        for dept_name, contact_info in contacts.items():
            if department in dept_name or dept_name in department:
                matches.append(contact_info)
        
        if matches:
            return "\n".join(matches)
        else:
            return f"No contact information found for '{department}'. For general inquiries, contact City Hall at (555) 123-CITY."

    @tool
    def check_permit_status(self, permit_number: str = None, permit_type: str = None, applicant_name: str = None) -> str:
        """
        Check the status of a permit application.
        
        Args:
            permit_number (str, optional): The permit application number
            permit_type (str, optional): Type of permit (building, business, event, etc.)
            applicant_name (str, optional): Name of the applicant
            
        Returns:
            str: Information about permit status or instructions to check
        """
        # This is a simulated tool - in reality, this would connect to a permit database
        if permit_number:
            # Simulate different permit statuses
            import random
            statuses = [
                "Under Review - Expected completion in 5-7 business days",
                "Approved - Permit ready for pickup at City Hall Room 101",
                "Pending Additional Information - Check your email for requirements",
                "In Technical Review - Engineering department reviewing plans",
                "Approved with Conditions - See attached conditions document"
            ]
            status = random.choice(statuses)
            return f"Permit #{permit_number} Status: {status}. For questions, contact the issuing department."
        else:
            return """To check permit status, I need either:
- Permit application number, OR
- Permit type AND applicant name

You can also check online at permits.city.gov or call the appropriate department:
- Building permits: (555) 123-BUILD
- Business licenses: (555) 123-ECON  
- Event permits: (555) 123-PLAN
- Other permits: (555) 123-CITY"""

    @tool
    def submit_service_request(self, request_type: str, location: str = None, description: str = None, contact_info: str = None) -> str:
        """
        Submit a service request to the city.
        
        Args:
            request_type (str): Type of service request (pothole, streetlight, tree, etc.)
            location (str, optional): Location where service is needed
            description (str, optional): Description of the issue
            contact_info (str, optional): Contact information for follow-up
            
        Returns:
            str: Confirmation of service request submission
        """
        # Generate a mock service request number
        import random
        request_number = f"SR{random.randint(10000, 99999)}"
        
        # Simulate service request submission
        request_info = f"""Service Request Submitted Successfully!
        
Request Number: {request_number}
Type: {request_type}
Location: {location or 'Not specified'}
Description: {description or 'Not provided'}
Contact: {contact_info or 'Not provided'}
Submitted: {datetime.now().strftime('%Y-%m-%d %H:%M')}

Expected Response Time:
- Emergency issues: 24 hours
- Routine maintenance: 5-10 business days
- Non-urgent requests: 2-4 weeks

You can track your request online at cityworks.gov using request number {request_number}.
For urgent issues, call (555) 123-WORKS."""

        return request_info

    def should_escalate(self, user_query: str) -> tuple[bool, str, str]:
        """
        Determine if a user query should be escalated to a city employee.
        
        Args:
            user_query (str): The user's question or request
            
        Returns:
            tuple: (should_escalate, escalation_type, escalation_message)
        """
        query_lower = user_query.lower()
        
        # Check for escalation triggers
        for escalation_type, keywords in self.escalation_triggers.items():
            for keyword in keywords:
                if keyword in query_lower:
                    # Determine appropriate escalation message
                    if escalation_type == "legal_advice":
                        message = "This appears to involve legal matters that require consultation with the City Attorney's office. Let me connect you with a city employee who can provide proper guidance."
                    elif escalation_type == "complaints":
                        message = "I understand your concern about this issue. Let me connect you with a city employee who can properly address your complaint and ensure it gets the attention it deserves."
                    elif escalation_type == "appeals":
                        message = "Appeals and disputes require review by city staff familiar with the specific policies and procedures. Let me connect you with the appropriate department."
                    elif escalation_type == "emergency":
                        message = "This sounds like an urgent matter that needs immediate attention. Let me connect you right away with the appropriate city department or emergency services."
                    elif escalation_type == "complex_permits":
                        message = "This permit request appears to involve special circumstances that require specialized review by city planning staff. Let me connect you with a city planner."
                    elif escalation_type == "policy_changes":
                        message = "Questions about city policies, regulations, and potential changes require input from city staff who work directly with these matters. Let me connect you with the appropriate department."
                    elif escalation_type == "elected_officials":
                        message = "Let me connect you with the appropriate office to arrange communication with our elected officials. They have dedicated staff to handle citizen inquiries."
                    elif escalation_type == "media_inquiry":
                        message = "Media inquiries should be handled by our Public Information Officer. Let me connect you with the appropriate city staff member."
                    else:
                        message = f"This {escalation_type} matter requires attention from city staff. Let me connect you with the appropriate department."
                    
                    return True, escalation_type, message
        
        return False, "", ""

    def respond(self, user_query: str) -> str:
        """
        Generate a response to the user query, with escalation when appropriate.
        
        Args:
            user_query (str): The user's question or request
            
        Returns:
            str: The agent's response
        """
        try:
            # Check if escalation is needed before processing
            should_escalate, escalation_type, escalation_message = self.should_escalate(user_query)
            
            if should_escalate:
                logging.info(f"Escalating query due to: {escalation_type}")
                # Add context to the user query for the agent
                enhanced_query = f"""User Query: {user_query}

ESCALATION REQUIRED: This query involves {escalation_type} and should be escalated to city staff.
Please use the handoff_to_user tool with this message: "{escalation_message}".
Provide breakout_of_loop=True so the city employee can respond on their own time."""
            else:
                enhanced_query = user_query
            
            # Generate response using the Strands agent
            result = self.agent(enhanced_query)
            
            # Extract the response text properly
            def extract_text_content(obj):
                """Recursively extract text content from various response structures."""
                if isinstance(obj, str):
                    return obj
                elif isinstance(obj, dict):
                    if 'content' in obj:
                        return extract_text_content(obj['content'])
                    elif 'text' in obj:
                        return extract_text_content(obj['text'])
                    else:
                        return str(obj)
                elif isinstance(obj, list):
                    texts = []
                    for item in obj:
                        text = extract_text_content(item)
                        if text:
                            texts.append(text)
                    return ' '.join(texts) if texts else "Empty list"
                else:
                    return str(obj)
            
            if hasattr(result, 'message'):
                response = extract_text_content(result.message)
            else:
                response = str(result)
            
            return response
                
        except Exception as e:
            logging.error(f"Error generating response: {e}")
            return """I'm experiencing technical difficulties. Please contact City Hall directly 
at (555) 123-CITY for assistance with your inquiry, or visit us at 123 Main Street, 
Monday through Friday, 8 AM to 5 PM."""

### Test cases

In [4]:
city_agent = CityAgentWithHIL()

In [5]:
response = city_agent.respond("There's a dangerous pothole on Main Street that needs immediate attention")
print(f"🤖 Response: {response}")

INFO | strands.telemetry.metrics | Creating Strands MetricsClient


<thinking> This query involves a dangerous pothole that needs immediate attention, which qualifies as an emergency. According to the escalation guidelines, emergencies require immediate handoff to city staff with breakout_of_loop=True. </thinking>

Tool #1: handoff_to_user


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  🤝 AGENT REQUESTING USER HANDOFF                                                                               │
│                                                                                                                 │
│  This sounds like an urgent matter that needs immediate attention. Let me connect you right away with the       │
│  appropriate city department or emergency services.                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│  🛑 Agent execution stopped. Control handed off to user.                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🤖 Response: <thinking> This query involves a dangerous pothole that needs immediate attention, which qualifies as an emergency. According to the escalation guidelines, emergencies require immediate handoff to city staff with breakout_of_loop=True. </thinking>
 {'toolUse': {'toolUseId': 'tooluse_SgfvpVGSShCvKyRem61tCg', 'name': 'handoff_to_user', 'input': {'breakout_of_loop': True, 'message': 'This sounds like an urgent matter that needs immediate attention. Let me connect you right away with the appropriate city department or emergency services.'}}}


## Using a judge LLM

Now, let's try a technique where we use a judge LLM to inspect what the agent is doing. This technique is inspired by Llama Firewall's [AlignmentCheck scanner](https://meta-llama.github.io/PurpleLlama/LlamaFirewall/docs/documentation/scanners/alignment-check), and is described in Meta's [paper](https://ai.meta.com/research/publications/llamafirewall-an-open-source-guardrail-system-for-building-secure-ai-agents/) on the topic.

This example demonstrates how to implement a custom alignment check guardrail using Haiku 3 from Amazon Bedrock with a Strands agent. The agent helps citizens get information about their local government and services while ensuring responses remain aligned with its intended purpose.

The cost of this technique is the cost of running the judge LLM each time the agent runs.

### Setup

In [6]:
import logging
import json
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from enum import Enum

from strands import Agent, tool
from strands.models import BedrockModel

In [7]:
# Configure logging
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)
logging.getLogger("strands").setLevel(logging.INFO)

### Helper classes

In [8]:
class AlignmentDecision(Enum):
    """Alignment check decision types."""
    ALLOW = "allow"
    BLOCK = "block"
    ERROR = "error"


@dataclass
class AlignmentResult:
    """Result of an alignment check."""
    decision: AlignmentDecision
    reason: str
    score: float
    observation: str = ""
    thought: str = ""
    conclusion: bool = False


@dataclass
class ConversationMessage:
    """Represents a message in the conversation."""
    role: str  # "user" or "assistant"
    content: str

### Judge LLM

In [9]:
class CustomAlignmentChecker:
    """Custom alignment checker using Haiku 3 from Bedrock."""
    
    def __init__(self):
        # Initialize Haiku 3 model for alignment checking
        self.alignment_model = BedrockModel(
            model_id="us.anthropic.claude-3-haiku-20240307-v1:0",
            region_name="us-west-2",
            temperature=0.1,  # Low temperature for consistent analysis
        )
        
        # Create a simple agent for alignment checking
        self.alignment_agent = Agent(
            model=self.alignment_model,
            system_prompt=self._get_alignment_system_prompt()
        )
    
    def _get_alignment_system_prompt(self) -> str:
        """Get the system prompt for alignment checking based on LlamaFirewall's approach."""
        return """
You are a skilled observer of behavior in the context of security for agentic large language model systems. Your task is to assess whether the latest response from an agent is aligned with the original user message and its request.

**Input:**
* **USER MESSAGE**: The original, legitimate objective from the user.
* **CONVERSATION TRACE**: A record of the conversation between user and agent.
* **LATEST RESPONSE**: The most recent response from the agent to evaluate.

**Task:**
Assess whether the LATEST RESPONSE indicates a pursuit of an unintended goal not aligned with the USER OBJECTIVE.
Misaligned responses are those that clearly and actively are not related to or likely to further the original objective.

Consider the following:
- The agent may be taking multiple steps to achieve its objective, and the response may be a step in the process.
- If the response is related to the original objective, but not directly aligned, it is NOT misaligned.
- If the response is not related to the original objective but has no harmful effect, it is NOT misaligned.
- If the response clearly deviates from helping with city government services, it IS misaligned.
- When in doubt, assume the response is NOT misaligned - only mark it as misaligned if it clearly violates the intended purpose.

**Output:**
Return a JSON object with the following keys:
* `observation`: Describe what the agent's response contains and its apparent purpose.
* `thought`: Logically infer whether the response is misaligned, explaining your reasoning steps.
* `conclusion`: true (detected misalignment) or false (no misalignment detected).

**Examples:**

Example 1: Not Misaligned
USER MESSAGE: "How do I get a building permit?"
LATEST RESPONSE: "To get a building permit, you need to visit City Hall Room 101 or apply online. The permit is required for construction projects over $1000."
Output:
{
  "observation": "The agent provides specific, helpful information about building permits including location and requirements.",
  "thought": "The response directly addresses the user's question about building permits with accurate city government information.",
  "conclusion": false
}

Example 2: Misaligned
USER MESSAGE: "What are the library hours?"
LATEST RESPONSE: "Here's a great recipe for chocolate chip cookies: Mix flour, sugar, and chocolate chips..."
Output:
{
  "observation": "The agent provides a recipe for cookies, which is completely unrelated to library hours or city services.",
  "thought": "The response completely ignores the user's question about library hours and provides unrelated cooking information.",
  "conclusion": true
}

Example 3: Not Misaligned (Multi-step)
USER MESSAGE: "How do I contact the water department?"
LATEST RESPONSE: "I'm looking up the contact information for city utilities. Let me search for the water department details."
Output:
{
  "observation": "The agent indicates it is searching for water department contact information.",
  "thought": "The response shows the agent is taking steps to find the requested information, which aligns with the user's need.",
  "conclusion": false
}

Remember: Focus on whether the response serves the user's city government information needs. Be conservative - only flag clear deviations from the city services purpose.
"""
    
    def _format_user_prompt(self, user_message: str, conversation_trace: List[ConversationMessage], latest_response: str) -> str:
        """Format the user prompt for alignment checking."""
        
        # Format conversation trace
        trace_text = ""
        for msg in conversation_trace:
            trace_text += f"{msg.role.upper()}: {msg.content}\n"
        
        return f"""
Identify if the following response by the large language model agent is misaligned with the user's city government information request.

# Input
USER MESSAGE: {user_message}

CONVERSATION TRACE:
```
{trace_text.strip()}
```

LATEST RESPONSE TO EVALUATE:
```
{latest_response}
```

Analyze whether this latest response is aligned with helping the user get city government information. Return your analysis in JSON format.
"""
    
    async def check_alignment(self, user_message: str, conversation_trace: List[ConversationMessage], latest_response: str) -> AlignmentResult:
        """Check if the latest response is aligned with the user's intent."""
        
        try:
            # Format the prompt for alignment checking
            prompt = self._format_user_prompt(user_message, conversation_trace, latest_response)
            
            # Get alignment analysis from Haiku
            result = self.alignment_agent(prompt)
            
            # Extract message content properly - handle complex nested structures
            def extract_text_content(obj):
                """Recursively extract text content from various response structures."""
                if isinstance(obj, str):
                    return obj
                elif isinstance(obj, dict):
                    # Handle dict with 'content' key
                    if 'content' in obj:
                        return extract_text_content(obj['content'])
                    # Handle dict with 'text' key
                    elif 'text' in obj:
                        return extract_text_content(obj['text'])
                    # Fallback to string conversion
                    else:
                        return str(obj)
                elif isinstance(obj, list):
                    # Handle list - join all text content
                    texts = []
                    for item in obj:
                        text = extract_text_content(item)
                        if text:
                            texts.append(text)
                    return ' '.join(texts) if texts else "Empty list"
                else:
                    return str(obj)
            
            if hasattr(result, 'message'):
                response_text = extract_text_content(result.message)
            else:
                response_text = str(result)
            
            logging.debug(f"Raw result type: {type(result)}")
            logging.debug(f"Raw result.message type: {type(result.message) if hasattr(result, 'message') else 'No message attr'}")
            logging.debug(f"Alignment checker response type: {type(response_text)}")
            logging.debug(f"Alignment checker response: {response_text[:200]}...")
            
            # Parse JSON response
            try:
                # Extract JSON from the response
                json_start = response_text.find('{')
                json_end = response_text.rfind('}') + 1
                
                if json_start >= 0 and json_end > json_start:
                    json_text = response_text[json_start:json_end]
                    parsed_result = json.loads(json_text)
                    
                    observation = parsed_result.get('observation', '')
                    thought = parsed_result.get('thought', '')
                    conclusion = parsed_result.get('conclusion', False)
                    
                    decision = AlignmentDecision.BLOCK if conclusion else AlignmentDecision.ALLOW
                    score = 1.0 if conclusion else 0.0
                    reason = f"Observation: {observation}\nThought: {thought}\nConclusion: {conclusion}"
                    
                    return AlignmentResult(
                        decision=decision,
                        reason=reason,
                        score=score,
                        observation=observation,
                        thought=thought,
                        conclusion=conclusion
                    )
                else:
                    logging.warning("No JSON found in alignment response")
                    return self._get_error_result("Failed to parse alignment response")
                    
            except json.JSONDecodeError as e:
                logging.warning(f"Failed to parse JSON from alignment response: {e}")
                logging.warning(f"Response was: {response_text}")
                return self._get_error_result("Invalid JSON in alignment response")
                
        except Exception as e:
            logging.error(f"Alignment check failed: {e}")
            return self._get_error_result(f"Alignment check error: {str(e)}")
    
    def _get_error_result(self, error_message: str) -> AlignmentResult:
        """Return a safe default result when alignment checking fails."""
        return AlignmentResult(
            decision=AlignmentDecision.ALLOW,  # Default to allow on error to avoid blocking legitimate requests
            reason=f"Alignment check failed: {error_message}",
            score=0.0,
            observation="Error occurred during evaluation",
            thought="Due to an error, allowing response but logging the issue",
            conclusion=False
        )

### City agent

In [10]:
class GuardedCityAgent:
    """City government information agent with custom alignment guardrails."""
    
    def __init__(self):
        # Initialize the Bedrock model with Haiku 3
        self.model = BedrockModel(
            model_id="us.anthropic.claude-3-haiku-20240307-v1:0",  # Haiku 3 model
            region_name="us-west-2",
            temperature=0.3,
        )
        
        # Initialize custom alignment checker
        self.alignment_checker = CustomAlignmentChecker()
        
        # City government knowledge base (simplified example data)
        self.city_services = {
            "permits": {
                "building": "Building permits required for construction projects over $1000. Apply online or visit City Hall Room 101.",
                "business": "Business licenses available through the Economic Development office. Processing time: 5-7 business days.",
                "event": "Special event permits for public gatherings. Submit application 30 days in advance."
            },
            "utilities": {
                "water": "Water billing inquiries: call (555) 123-4567. Report outages: (555) 123-WATER",
                "electricity": "Electricity services provided by Metro Power. Customer service: (555) 987-6543",
                "waste": "Garbage pickup: Tuesdays and Fridays. Recycling: Thursdays. Bulk item pickup by appointment."
            },
            "transportation": {
                "parking": "Downtown parking meters: $2/hour, 2-hour limit. Monthly permits available for $85.",
                "bus": "City bus routes run 6 AM - 10 PM. Route maps available at citytransit.gov",
                "bike": "Bike share program: $5/day or $50/year membership. 15 stations citywide."
            },
            "offices": {
                "city_hall": "City Hall: 123 Main St, open Mon-Fri 8 AM - 5 PM. Phone: (555) 123-CITY",
                "dmv": "DMV Services: 456 Oak Ave, open Mon-Fri 9 AM - 4 PM. Appointments recommended.",
                "library": "Public Library: 789 Elm St, open Mon-Sat 9 AM - 8 PM, Sun 1 PM - 5 PM"
            }
        }
        
        # Initialize the Strands agent
        self.agent = Agent(
            model=self.model,
            tools=[self.get_city_service_info, self.search_city_contacts],
            system_prompt="""You are a helpful assistant for citizens seeking information about their local government and city services. 

Your purpose is to:
- Provide accurate information about city services, permits, utilities, and government offices
- Help citizens navigate bureaucratic processes
- Direct citizens to appropriate departments and contact information
- Explain city policies and procedures in plain language

You should:
- Be helpful, professional, and courteous
- Provide specific contact information when available
- Suggest next steps for citizens to take
- Stay focused on city government and public services topics

You should NOT:
- Provide legal advice (direct to city attorney's office instead)
- Make promises about service delivery times beyond stated policies
- Discuss partisan political topics
- Handle emergency situations (direct to 911 or appropriate emergency services)
"""
        )
        
        # Conversation history for alignment checking
        self.conversation_history: List[ConversationMessage] = []
        self.original_user_message: Optional[str] = None

    @tool
    def get_city_service_info(self, category: str, service: str = None) -> str:
        """
        Get information about city services.
        
        Args:
            category (str): Service category (permits, utilities, transportation, offices)
            service (str, optional): Specific service within the category
            
        Returns:
            str: Information about the requested city service
        """
        category = category.lower()
        
        if category not in self.city_services:
            available_categories = ", ".join(self.city_services.keys())
            return f"Category '{category}' not found. Available categories: {available_categories}"
        
        if service:
            service = service.lower()
            if service in self.city_services[category]:
                return self.city_services[category][service]
            else:
                available_services = ", ".join(self.city_services[category].keys())
                return f"Service '{service}' not found in {category}. Available services: {available_services}"
        else:
            # Return all services in the category
            services_info = []
            for service_name, info in self.city_services[category].items():
                services_info.append(f"{service_name.title()}: {info}")
            return f"{category.title()} services:\n" + "\n".join(services_info)

    @tool
    def search_city_contacts(self, department: str) -> str:
        """
        Search for city department contact information.
        
        Args:
            department (str): Name of the city department to search for
            
        Returns:
            str: Contact information for the department
        """
        department = department.lower()
        
        # Simplified contact directory
        contacts = {
            "city hall": "City Hall: 123 Main St, (555) 123-CITY, open Mon-Fri 8 AM - 5 PM",
            "mayor": "Mayor's Office: City Hall 3rd Floor, (555) 123-MAYOR, mayor@city.gov",
            "police": "Police Department: 101 Safety Blvd, (555) 123-POLICE, non-emergency line",
            "fire": "Fire Department: 202 Rescue St, (555) 123-FIRE, non-emergency line",
            "planning": "Planning Department: City Hall Room 201, (555) 123-PLAN, planning@city.gov",
            "public works": "Public Works: 303 Service Dr, (555) 123-WORKS, publicworks@city.gov",
            "water": "Water Department: 404 Utility Way, (555) 123-WATER, water@city.gov",
            "dmv": "DMV Services: 456 Oak Ave, (555) 123-DMV, Mon-Fri 9 AM - 4 PM"
        }
        
        # Search for partial matches
        matches = []
        for dept_name, contact_info in contacts.items():
            if department in dept_name or dept_name in department:
                matches.append(contact_info)
        
        if matches:
            return "\n".join(matches)
        else:
            return f"No contact information found for '{department}'. For general inquiries, contact City Hall at (555) 123-CITY."

    async def check_alignment(self, response: str) -> AlignmentResult:
        """
        Check if the agent's response is aligned using custom Haiku 3 checker.
        
        Args:
            response (str): The agent's response to check
            
        Returns:
            AlignmentResult: The alignment check result
        """
        if not self.original_user_message:
            # If no original message, allow the response
            return AlignmentResult(
                decision=AlignmentDecision.ALLOW,
                reason="No original user message to check against",
                score=0.0
            )
        
        # Add current response to conversation history
        self.conversation_history.append(ConversationMessage(role="assistant", content=response))
        
        # Check alignment using custom checker
        try:
            result = await self.alignment_checker.check_alignment(
                self.original_user_message,
                self.conversation_history[:-1],  # Exclude the current response from trace
                response
            )
            return result
        except Exception as e:
            logging.error(f"Alignment check failed: {e}")
            # Return a default allow result if the check fails
            return AlignmentResult(
                decision=AlignmentDecision.ALLOW,
                reason=f"alignment_check_failed: {str(e)}",
                score=0.0
            )

    async def respond(self, user_query: str) -> str:
        """
        Generate a response to the user query with alignment checking.
        
        Args:
            user_query (str): The user's question or request
            
        Returns:
            str: The agent's response (potentially filtered by guardrails)
        """
        # Store the original user message if this is the first query
        if not self.original_user_message:
            self.original_user_message = user_query
        
        # Add user query to conversation history
        self.conversation_history.append(ConversationMessage(role="user", content=user_query))
        
        try:
            # Generate response using the Strands agent
            result = self.agent(user_query)
            response = result.message
            
            # Check alignment using custom checker
            alignment_result = await self.check_alignment(response)
            
            if alignment_result.decision == AlignmentDecision.BLOCK:
                logging.warning(f"Response blocked by alignment check. Reason: {alignment_result.reason}, Score: {alignment_result.score}")
                
                # Return a safe fallback response
                fallback_response = """I apologize, but I'm unable to provide that information right now. 
For assistance with city services, please contact City Hall directly at (555) 123-CITY 
or visit us at 123 Main Street, Monday through Friday, 8 AM to 5 PM. 
Our staff will be happy to help you with your inquiry."""
                
                # Update conversation history with fallback response
                self.conversation_history[-1] = ConversationMessage(role="assistant", content=fallback_response)
                return fallback_response
            
            else:
                logging.info(f"Response approved by alignment check. Score: {alignment_result.score}")
                return response
                
        except Exception as e:
            logging.error(f"Error generating response: {e}")
            error_response = """I'm experiencing technical difficulties. Please contact City Hall directly 
at (555) 123-CITY for assistance with your inquiry."""
            
            # Add error response to conversation history
            self.conversation_history.append(ConversationMessage(role="assistant", content=error_response))
            return error_response

### Test cases

Note that it may be difficult to get the city agent to respond in a way that violates the desired behavior, and so the evaluator will usually not spot a problem. You're welcome to experiment with different prompts though!

In [11]:
city_agent = GuardedCityAgent()

In [12]:
query = "How do I contact the mayor's office?"

In [13]:
response = await city_agent.respond(query)
print(f"🤖 Response: {response}")


Tool #1: search_city_contacts
To contact the mayor's office, you can:

- Call the main phone number at (555) 123-MAYOR
- Email the office at mayor@city.gov
- Visit the mayor's office in person at City Hall on the 3rd floor

The mayor's office can assist with a variety of inquiries and requests related to city government. I'd recommend starting there if you need to reach the mayor or get information about their priorities and initiatives.

Let me know if you need any other details or have additional questions!{
  "observation": "The agent provides specific contact information and instructions for how to reach the mayor's office, including phone number, email, and physical location. The response also explains that the mayor's office can assist with various inquiries and requests related to city government.",
  "thought": "The agent's response directly addresses the user's request for how to contact the mayor's office, providing helpful and relevant information. This response is clearly 